# Example 5: Running non-Poissonian Scans with MultiNest

In this example we show a simple example of how to run a scan using templates that follow non-Poisson statistics.

To this end we perform a simple analysis of a small region around the north galactic pole, looking for isotropically distributed point sources.

**NB:** this example makes use of the Fermi Data, which needs to already be installed. See Example 1 for details.

In [1]:
# Import relevant modules

%matplotlib inline
%load_ext autoreload
%autoreload 2

import numpy as np
import corner
import matplotlib.pyplot as plt
from matplotlib import rcParams

from NPTFit import nptfit # module for performing scan
from NPTFit import create_mask as cm # module for creating the mask
from NPTFit import psf_correction as pc # module for determining the PSF correction
from NPTFit import dnds_analysis # module for analysing the output

In [2]:
# Set plotting options
rcParams['xtick.labelsize'] = 20
rcParams['ytick.labelsize'] = 20
rcParams['axes.labelsize'] = 20
rcParams['axes.titlesize'] = 20
rcParams['font.family'] = 'serif'
rcParams['font.serif'] = 'CMU Serif'
rcParams['figure.figsize'] = (7,5)
rcParams['legend.fontsize'] = 16
# rcParams['text.usetex'] = True

## Step 1: Setup an instance of NPTFit, add data, mask and templates

As in the Poissonian case (see Example 3), the first step is to create an instance of `nptfit.NPTF` and add to it data, a mask and templates. Here we choose an ROI that is a small ring around the galactic north pole. In this small region we will only use isotropically distributed templates.

In [3]:
n = nptfit.NPTF(tag='non-Poissonian_Example')

In [4]:
fermi_data = np.load('fermi_data/fermidata_counts.npy')
fermi_exposure = np.load('fermi_data/fermidata_exposure.npy')
n.load_data(fermi_data, fermi_exposure)

In [5]:
analysis_mask = cm.make_mask_total(mask_ring = True, inner = 0, outer = 5, ring_b = 90, ring_l = 0)
n.load_mask(analysis_mask)

In [6]:
iso = np.load('fermi_data/template_iso.npy')
n.add_template(iso, 'iso')

## Step 2: Add Background Models to the Fit

We now add background models. For the details of adding a Poissonian model, see Example 3.

For the non-poissonian model the format is similar, except for the fact we must account for the additional parameters that describe an non-Poissonian (NP) template. In `nptfit`, NP templates are determined by source count functions which we allow to be specified by a broken power law with an arbitrary number of breaks. 

The simplest possible example consistent with a finite number of sources is a singly broken power law, which is specified by four parameters: the template normalisation $A$, indices above and below the break $n_1$ and $n_2$, and the location of the break $S_b$.

In general, for a source count function with $\ell$ breaks, the $2\ell+2$ parameters are specified as follows:

$$\left[ A, n_1, \ldots, n_{\ell+1}, S_b^{(1)}, \ldots, S_b^{(\ell)} \right]\,,$$

where $n_1$ is the highest index and $S_b^{(1)}$ the highest break.

Priors must be entered as an array where each element is an array of the priors for each unfixed parameter. For multiply broken power laws it is possible to specify the breaks in terms of the highest break, in which case the option `dnds_model=specifiy_relative_breaks` should be used.

Fixed parameters are similarly entered as an array, where the first element is the location of the parameter to be fixed (an integer), and the second element is the value to which it should be fixed.

In the example below we add an isotropic distributed non-Poissonian template, with a log flat normalisation, linear flat indices, and a fixed break.

In [7]:
n.add_poiss_model('iso','$A_\mathrm{iso}$', False, fixed=True, fixed_norm=1.47)
n.add_non_poiss_model('iso',
                      ['$A^\mathrm{ps}_\mathrm{iso}$','$n_1$','$n_2$','$S_b$'],
                      [[-6,1],[2.05,30],[-2,1.95]],
                      [True,False,False],
                      fixed_params = [[3,22.]])

## Step 3: Configure the Scan with the PSF correction

For a non-Poissonian fit, we need to specify the PSF correction at the stage of configuring the scan. The details of this are described in Example 4. These are calculated using `psf_correction.py` and then passed to the `NPTF` via `configure_for_scan`.

At this stage we also specify the number of exposure regions to be used. Here we take `nexp=1` for a simple example. Generally increasing `nexp` leads to more accurate results, but also increases the runtime of the code. 

In [ ]:
pc_inst = pc.PSFCorrection(psf_sigma_deg=0.1812)
f_ary = pc_inst.f_ary
df_rho_div_f_ary = pc_inst.df_rho_div_f_ary

n.configure_for_scan(f_ary=f_ary, df_rho_div_f_ary=df_rho_div_f_ary, nexp=1)

Loading the psf correction from: /group/hepheno/smsharma/NPTFit/examples/psf_dir/gauss_128_0.181_10_50000_1000_0.01.npy
The number of parameters to be fit is 3


## Step 4: Perform the Scan

Next we perform the scan. The syntax is identical to the Poissonian case and described in Example 3. Note even though we float fewer parameters than in the Poissonian example, the runtime is longer here. This is due to the fact that the NPTF likelihood is inherently more complicated and so takes longer to evaluate.

In [ ]:
n.perform_scan(nlive=500)

[['iso', 2.3900887893529515e-06], ['iso', 15.347284948825834], ['iso', 0.21919571161270168], ['iso', 22.0]]
[[2.39008878935e-06, 15.3472849488, 0.219195711613, 22.0]]
[[2.39008878935e-06, 15.3472849488, 0.219195711613, 22.0]]
[[2.39008878935e-06, 15.3472849488, 0.219195711613, 22.0]]
[['iso', 1.2984308260291324e-06], ['iso', 10.729445302486418], ['iso', 1.4500104129314426], ['iso', 22.0]]
[[1.2984308260299999e-06, 10.7294453025, 1.45001041293, 22.0]]
[[1.2984308260299999e-06, 10.7294453025, 1.45001041293, 22.0]]
[[1.2984308260299999e-06, 10.7294453025, 1.45001041293, 22.0]]
[['iso', 5.5880682663951525e-05], ['iso', 12.895606553554533], ['iso', -1.4199046343564987], ['iso', 22.0]]
[[5.5880682664000002e-05, 12.8956065536, -1.4199046343599999, 22.0]]
[[5.5880682664000002e-05, 12.8956065536, -1.4199046343599999, 22.0]]
[[5.5880682664000002e-05, 12.8956065536, -1.4199046343599999, 22.0]]
[['iso', 0.05895357375485584], ['iso', 5.885809481143951], ['iso', 0.3311824321746828], ['iso', 22.0]]
[

/group/hepheno/heptools/anaconda3/envs/venv_py27/lib/python2.7/site-packages/NPTFit-0.1.dev0-py2.7-linux-x86_64.egg/NPTFit/nptf_scan.py:416: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future


[['iso', 0.0001280039032016659], ['iso', 21.973123902082442], ['iso', -1.1584249228239059], ['iso', 22.0]]
[[0.00012800390320199999, 21.973123902099999, -1.1584249228200001, 22.0]]
[[0.00012800390320199999, 21.973123902099999, -1.1584249228200001, 22.0]]
[[0.00012800390320199999, 21.973123902099999, -1.1584249228200001, 22.0]]
[['iso', 0.001204183270346112], ['iso', 16.781236264109612], ['iso', -0.17825670838356], ['iso', 22.0]]
[[0.0012041832703499999, 16.781236264099999, -0.17825670838400001, 22.0]]
[[0.0012041832703499999, 16.781236264099999, -0.17825670838400001, 22.0]]
[[0.0012041832703499999, 16.781236264099999, -0.17825670838400001, 22.0]]
[['iso', 0.03682760142519851], ['iso', 3.5846646457910536], ['iso', -1.7632269531488418], ['iso', 22.0]]
[[0.036827601425200002, 3.5846646457900002, -1.76322695315, 22.0]]
[[0.036827601425199995, 3.5846646457900002, -1.76322695315, 22.0]]
[[0.036827601425199995, 3.5846646457900002, -1.76322695315, 22.0]]
[['iso', 0.0005499762409747633], ['iso'

ERROR: Interrupt received: Terminating


## Step 5: Analyze the Output

Here we analyze the output using the same commands as in the Poissonian example.

In [ ]:
n.load_scan()
cs=dnds_analysis.Analysis(n)
cs.make_triangle()

We also show a plot of the source count function, although a careful explanation of the details here are deferred until Example 8.

In [ ]:
cs.plot_source_count_median('iso',smin=0.01,smax=10000,nsteps=1000,spow=2,color='forestgreen')
cs.plot_source_count_band('iso',smin=0.01,smax=10000,nsteps=1000,qs=[0.16,0.5,0.84],spow=2,color='forestgreen',alpha=0.3)

plt.yscale('log')
plt.xscale('log')
plt.xlim([1e-11,1e-7])
plt.ylim([1e-16,1e-10])
plt.tick_params(axis='x', length=5,width=2,labelsize=18)
plt.tick_params(axis='y',length=5,width=2,labelsize=18)
plt.xlabel('$F$  [counts/cm$^2$/s]', fontsize=18)
plt.ylabel('$F^2 dN/dF$ [counts/cm$^2$/s/deg$^2$]', fontsize=18)